In [3]:
pip install statsmodels

In [4]:
import pandas as pd
import sys,codecs
import numpy as np
import re
import os
import matplotlib.pyplot as plt
import scipy.stats as st
import collections
import statsmodels.api as sm
from time import time
import random

In [3]:
import os

NUM_CPU = len(os.sched_getaffinity(0)) #os.cpu_count() 

print(f'CPU total: {NUM_CPU}')

CPU total: 36


In [4]:
NUM_THREADS = 4

os.environ["MKL_NUM_THREADS"]     = str(NUM_THREADS)
os.environ["NUMEXPR_NUM_THREADS"] = str(NUM_THREADS)
os.environ["OMP_NUM_THREADS"]     = str(NUM_THREADS)

NUM_PROCESS = NUM_CPU // NUM_THREADS
print(f'max process: {NUM_PROCESS}')

max process: 9


In [5]:
# dimensions
n = 500; p = 50; q = 20; d = 3

In [6]:
# true parameter
tau = np.round(np.random.uniform(0.15,0.2,p),4)
rho = np.round(np.random.uniform(0.2,0.9,p),4)
beta0 = np.round(np.random.uniform(0.5,1,(p,10)),4)
beta1 = np.zeros((p,q-10))
beta = np.hstack((beta0, beta1))
bc = np.array(np.hstack([np.random.normal(0,1,(p,d))]))
# Omega mean and variance
mean = np.zeros(p)
ta = 0.15
i, j = np.mgrid[:p, :p]
cov = ta**2*ta**abs(i-j)
cov[list(range(p)),list(range(p))] = ta
sigma2 = np.zeros(p)
for j in range(p):
    sigma2[j] = bc[j,:]@bc[j,:] + ta

In [7]:
# adjacency matrix-DIM
pv = [2/n,0.5*(n**(-0.8)),0.5*(n**(-0.8)),1-2/n-0.5*(n**(-0.8))-0.5*(n**(-0.8))]
A = np.zeros((n,n))
for i in range(0,n):
    for j in range(i+1,n):
        m = np.argmax(np.random.multinomial(1,pv,1))
        if m==0:
            A[i,j] = A[j,i] = 1
        elif m==1:
            A[i,j] = 1; A[j,i] = 0
        elif m==2:
            A[i,j] = 0; A[j,i] = 1
ind = (np.where(np.sum(A,1)==0))[0]
for i in ind:
    A[i,random.sample(list(range(n)),4)] = 1
A[list(range(n)),list(range(n))] = 0
W = A/np.sum(A,1).reshape(n,1)

In [8]:
# Check
print(np.sum(A))
np.max(np.sum(A,1))

1821.0


11.0

In [93]:
# save
pd.DataFrame(rho).to_csv("Results_dep_02/n500_p50_B500/rho_true_n500_p50.csv",index=False)
pd.DataFrame(beta).to_csv("Results_dep_02/n500_p50_B500/beta_true_n500_p50.csv",index=False)
pd.DataFrame(bc).to_csv("Results_dep_02/n500_p50_B500/B_true_n500_p50.csv",index=False)
pd.DataFrame(A).to_csv("Results_dep_02/A_smallp_n500.csv")

In [9]:
# Data generation function
def data_generator(nn, p, q, d, seed):

    rng = np.random.default_rng(seed) 
    
    q0 = 10
    Omega = rng.multivariate_normal(mean, cov, (nn,), 'raise')   
    X = rng.normal(0,1,(nn,q0))
    Z = rng.normal(0,1,(nn,d))
    
    Y = np.zeros((n,p))
    for j in range(p):
        y_j = np.dot(np.linalg.inv(np.eye(nn)-rho[j]*W), X@beta[j,:q0] + Z@bc[j,:]+Omega[:,j])#
        Y[:,j] = y_j 
        
    return Y,X,Z

In [10]:
# CMLE-gradient
def gardient_initial_lj(nn, p, q, WW, para, Yj, XX):
    
    q0 = 10
    
    rho_j = para[0]
    beta_j = para[1:(q0+1)]
    sigma2_j = para[-1]
    
    Sj = np.eye(nn)-rho_j*WW
    S_inverse = np.linalg.inv(Sj)
    G = WW@S_inverse
    Ep = Sj@Yj - XX@beta_j
    
    g1 = -np.trace(G)+np.dot((WW@Yj).T,Ep)/sigma2_j
    g2 = (XX.T@Ep)/sigma2_j
    g3 = -nn/(2*sigma2_j)+(Ep.T@Ep)/(2*(sigma2_j**2))
    
    return np.concatenate(([g1],g2,[g3]))

In [11]:
# CMLE-hessian
def hessian_initial_lj(nn, p, q, Wt, para, Yj, XX):
    
    q0 = 10
    
    #initial values
    rho_j = para[0]
    beta_j = para[1:(q0+1)]
    sigma2_j = para[-1]
    
    Sj = np.eye(nn)-rho_j*Wt
    S_inverse = np.linalg.inv(Sj)
    G = np.dot(Wt,S_inverse)
    WW = np.dot(Wt.T,Wt)
    WS = np.dot(Wt.T,Sj)
    SS = np.dot(Sj.T,Sj)
    Ep = Sj@Yj - XX@beta_j
    
    
    h11 = -np.trace(G@G)-np.dot(Yj.T@WW,Yj)/sigma2_j
    h12 = h21 = -(XX.T@(Wt@Yj)/sigma2_j).reshape(q0,1)
    h31 = h13 = -np.dot((Wt@Yj).T,Ep)/(sigma2_j**2)
    
    h22 = -XX.T@XX/sigma2_j
    h23 = h32 = -(XX.T@Ep/(2*sigma2_j**2)).reshape(q0,1)
    h33 = nn/(2*sigma2_j**2)-np.dot(Yj.T@SS,Yj)/(sigma2_j**3)
    
    H = np.block([
         [h11, h12.T, h13],
         [h21, h22, h23],
         [h31, h32.T, h33]
    ])
    
    return H

In [12]:
# Newton--CMLE
def newton_sea_initial(nn, p, q, Wt, pa0, Yj, XX, max_iter = 50, eps = 1e-4):
    
    q0 = 10
    pa_new = pa0
    for t in range(max_iter):
        pa_pre = pa_new
        gradient = gardient_initial_lj(nn, p, q, Wt, pa_pre, Yj, XX)/nn
        hessian =  hessian_initial_lj(nn, p, q, Wt, pa_pre, Yj, XX)/nn 
        diff = np.linalg.solve(hessian+0.001*np.eye(q0+2),gradient)
        pa_new = pa_pre - diff 
        if pa_new[-1]<0.01: pa_new[-1] = 0.01
        if pa_new[0]>1: pa_new[0] = 0.95
        #print(np.max(abs(diff)))
        if np.linalg.norm(diff) < eps:
            break
            
    return pa_new,t+1

In [13]:
# SAR optimization function
def optimize_initial(nn, p, q, Wt,YY, XX,bb):
    
    q0 = 10
    para = np.zeros((q0+2,p))
    ite = np.zeros(p)
    par = np.zeros(q0+2)
    par[-1] = 0.1
    for j in range(p):
        res = newton_sea_initial(nn, p, q, Wt, par, YY[:,j], XX)
        para[:,j] = res[0]
        if para[0,j]> 0.95: 
            para[0,j] = 0.95
        if para[0,j]< 0.05: 
            para[0,j] = 0.05
        ite[j] = res[1]
        with open('Results_dep_02/process.txt', 'a') as f1:
            f1.write(str(bb)+', initial: '+ str(j) + ', '+ str(para[0,j]) +'\n')
    
    return para[0,:],para[1:(q0+1),:], para[-1,:],ite

In [14]:
# FMLE-gradient
def gardient_lj(nn, p, q, d, Wt,the0, Z_ee, Yj, XX):
    
    q0 = 10
    rhoj = the0[0]
    betaj = the0[1:(q0+1)]
    bj = the0[(q0+1):(q0+d+1)]
    tauj2 = the0[-1]
    
    Sj = np.eye(nn)-rhoj*Wt
    S_inverse = np.linalg.inv(Sj)
    G = Wt@S_inverse
    Ep = Sj@Yj - XX@betaj - Z_ee@bj
    
    g1 = -np.trace(G)+np.dot((Wt@Yj).T,Ep)/tauj2
    g2 = (XX.T@Ep)/tauj2
    g3 = (Z_ee.T@Ep)/tauj2
    g4 = -nn/(2*tauj2)+(Ep.T@Ep)/(2*tauj2**2)
    
    return np.concatenate(([g1],g2,g3,[g4]))

In [15]:
# FMLE-hessian
def hessian_lj(nn, p, q, d, Wt, the0, Z_e, Yj, XX):
    
    q0 = 10
    rhoj = the0[0]
    betaj = the0[1:(q0+1)]
    bj = the0[(q0+1):(q0+d+1)]
    tauj2 = the0[-1]
    
    Sj = np.eye(nn)-rhoj*Wt
    S_inverse = np.linalg.inv(Sj)
    G = np.dot(Wt,S_inverse)
    WY = Wt@Yj
    Ep = Sj@Yj - XX@betaj - Z_e@bj
    
    h11 = -np.trace(G@G) - WY.T@WY/tauj2
    
    h12 = h21 = -(XX.T@WY/tauj2).reshape(q0,1)
    h13 = h31 = -(Z_e.T@WY/tauj2).reshape(d,1)
    h14 = h41 = -WY.T@Ep/tauj2**2
    
    h22 = -XX.T@XX/tauj2
    h23 = h32 = -XX.T@Z_e/tauj2
    h24 = h42 = -(XX.T@Ep/(2*tauj2**2)).reshape(q0,1)
    
    h33 = -Z_e.T@Z_e/tauj2
    h34 = h43 = -(Z_e.T@Ep/(2*tauj2**2)).reshape(d,1)
    
    h44 = nn/(2*tauj2**2)-Ep.T@Ep/(tauj2**3)
    
    H = np.block([
         [h11, h12.T, h13.T, h14],
         [h21, h22, h23, h24],
         [h31, h32.T, h33, h34],
         [h41, h42.T, h43.T, h44]
    ])
    
    return H

In [16]:
# Newton--FMLE
def newton_sea(nn, p, q, d, Wt, theta0, Z_e, Yj, Xt, max_iter = 50, eps = 1e-4):
    
    q0 = 10
    theta_new = theta0
    for t in range(max_iter):
        theta_pre = theta_new
        gradient = gardient_lj(nn, p,q,d,Wt, theta_pre, Z_e, Yj,Xt)/nn 
        hessian =  hessian_lj(nn, p,q,d,Wt, theta_pre, Z_e, Yj,Xt)/nn 
        diff = np.linalg.solve(hessian+0.001*np.eye(q0+d+2),gradient)
        #print(np.max(abs(diff)))
        theta_new = theta_pre - diff
        if theta_new[-1]<0.01: theta_new[-1] = 0.01
        if theta_new[0]>1: theta_new[0] = 0.95

        if np.linalg.norm(diff) < eps:
            break
            
    return theta_new,t+1

In [17]:
# FMLE optimization function
def optimize(nn, p, q, d, Wt, Z_e, Yt, Xt,bb):
    
    q0 = 10
    g = q0+d+2
    theta = np.zeros((g,p))
    itee = np.zeros(p)
    theta0 = np.zeros(g)
    theta0[-1] = 0.1
    for j in range(p):
        res = newton_sea(nn, p, q, d, Wt, theta0, Z_e, Yt[:,j], Xt)
        theta[:,j] = res[0]
        itee[j] = res[1]
        if theta[0,j]> 0.95: 
            theta[0,j] = 0.95
        if theta[0,j]< 0.05: 
            theta[0,j] = 0.05
        with open('Results_dep_02/process.txt', 'a') as f1:
            f1.write(str(bb)+', update: '+ str(j) + ', '+ str(theta[0,j]) +'\n')
    
    return theta[0,:],theta[1:(q0+1),:],theta[(q0+1):(q0+d+1),:], theta[-1,:], itee

In [18]:
# Factor estimation function
def Z_est(nn, p, q, d, Wt0, Wt1, Y_1, X_1, rho_e, beta_e, Yb, Xb, rho_hb, beta_hb):
    
    Eb = Yb - (Wt0@Yb)@np.diag(rho_hb) - Xb@beta_hb
    Sigm = Eb.T@Eb/nn 
    U, G, V = np.linalg.svd(Sigm)
    M = U[:,:d]@np.diag(G[:d])**0.5
    
    E = Y_1 - (Wt1@Y_1)@np.diag(rho_e) - X_1@beta_e
     
    Z_e = E@M/p
    
    return Z_e, M

In [19]:
# Confidence interval for FMLE
def CI_est_test(j,nn, p, q0, d, WW, rho_est, beta_est, b_est, tau2_est, B_hat, sigma2_hat, YY, XX, Z_est, z_alpha):
    
    # ticn1 = time()
    S = np.eye(nn)-rho_est[j]*WW
    S_inverse = np.linalg.inv(S)
    G = np.dot(WW,S_inverse)
    Gs = G + G.T
    GXbe = G@XX@beta_est[:,j]
    GZb = G@Z_est@b_est[:,j]

    c1 = tau2_est[j]*np.trace(G@Gs)
    c21 = GXbe + GZb
    C11 = c1 + c21.T@c21
    C12 = C21 = (XX.T@GXbe).reshape(q0,1)
    C13 = C31 = (Z_est.T@GZb).reshape(d,1)
    C14 = C41 = np.trace(G)
    C22 = XX.T@XX
    C23 = C32 = XX.T@Z_est
    C24 = C42 = np.zeros((q0,1))
    C33 = Z_est.T@Z_est
    C34 = C43 = np.zeros((d,1))
    C44 = nn/(2*tau2_est[j])
            
    Sigma2 = np.block([
             [C11, C12.T, C13.T, C14],
             [C21, C22, C23, C24],
             [C31, C32.T, C33, C34],
             [C41, C42.T, C43.T, C44],
        ])/(nn*tau2_est[j])
    
    Sigma2_inv = np.linalg.inv(Sigma2)
    # tocn1 = time()
    # print(tocn1 - ticn1) 
    
    g = q0+d+2
    Mbtj = B_hat@b_est[:,j]
    Omega_hat = S@YY[:,j] - XX@beta_est[:,j] - Z_est@b_est[:,j]
    Aj = np.zeros((nn*d,g*nn*d))
    Dj = np.zeros((nn*d,g))
    for k in range(p):
        Sk = np.eye(nn)-rho_est[k]*WW
        S_inverse = np.linalg.inv(Sk)
        G = np.dot(WW,S_inverse)
        GXbeta = G@XX@beta_est[:,k]
        
        C11 = np.trace(G@(G+G.T)) + GXbeta.T@GXbeta/sigma2_hat[k]
        C12 = C21 = (XX.T@GXbeta/sigma2_hat[k]).reshape(q0,1)
        C13 = C31 = np.trace(G)/sigma2_hat[k]
        C22 = XX.T@XX/sigma2_hat[k]
        C23 = C32 = np.zeros((q0,1))
        C33 = nn/(2*sigma2_hat[k]**2)
        
        Sigma = np.block([
             [C11, C12.T, C13],
             [C21, C22, C23],
             [C31, C32.T,C33]
        ])/nn
        H = np.linalg.inv(Sigma)
        H_rho = H[0,:]
        H_beta = H[1:-1,:]
    
        A2 = G.T/sigma2_hat[k]
        A3 = np.eye(nn)/(2*sigma2_hat[k]**2)
        A1 = H_rho[0]*A2 + H_rho[-1]*A3
        
    
        cjk_rho = YY[:,j].T@WW.T@WW@YY[:,k]*Mbtj[k]/nn
        cjk_beta = XX.T@WW@YY[:,k]*Mbtj[k]/nn
        cjk_b = Z_est.T@WW@YY[:,k]*Mbtj[k]/nn - B_hat[k,:]*(Omega_hat@WW@YY[:,k])/nn
        cjk_tau2 = Omega_hat@WW@YY[:,k]*Mbtj[k]/(nn*tau2_est[j])
        Qjk_rho = (XX.T@WW@YY[:,j]*Mbtj[k]/nn).reshape(q0,1)
        Qjk_beta = XX.T@XX*Mbtj[k]/nn # q0 c q0
        Qjk_b = XX.T@Z_est*Mbtj[k]/nn - (XX.T@Omega_hat).reshape(q0,1)@B_hat[k,:].reshape(1,d)/nn # q0 c d
        Qjk_tau2 = (XX.T@Omega_hat*Mbtj[k]/(nn*tau2_est[j])).reshape(q0,1)
        cjk_Theta = np.hstack([cjk_rho, cjk_beta, cjk_b, cjk_tau2]) #  g
        Qjk_Theta = np.concatenate([Qjk_rho, Qjk_beta, Qjk_b, Qjk_tau2], axis=1) # q0 c g
        
        vjk1 = cjk_Theta # g 
        vjk2 = Qjk_Theta.T@H_beta[:,0] # g 
        vjk3 = Qjk_Theta.T@H_beta[:,-1] # g 
        
        Ak = np.kron(A1, vjk1.reshape(1,g)) + np.kron(A2, vjk2.reshape(1,g)) + np.kron(A1, vjk3.reshape(1,g))
        Akb = np.kron(b_est[:,k].reshape(d,1)@b_est[:,k].reshape(1,d), Ak)
        Aj = Aj + Akb/p
    
        vjk = (cjk_Theta*H_rho[0] + Qjk_Theta.T@H_beta[:,0])/sigma2_hat[k] # g
        Ujk = (cjk_Theta.reshape(g,1)@H_beta[:,0].reshape(1,q0) + Qjk_Theta.T@H_beta[:,1:-1])/sigma2_hat[k] # g c q0
        Djk = GXbeta.reshape(nn,1)@vjk.reshape(1,g) + XX@Ujk.T # n c g
        Djkb = np.kron(b_est[:,k].reshape(d,1), Djk)
        Dj = Dj + Djkb/p

    # tocn1 = time()
    # print(tocn1 - ticn1)

    Sigma_Q = np.zeros((g, g))
    H_hat = B_hat.T @ B_hat / p
    HI = np.kron(H_hat @ H_hat.T, np.eye(nn))
    Z_hat = Z_est @ np.linalg.inv(H_hat)
    mu3z = np.mean(Z_hat**3)
    mu4z = np.mean(Z_hat**4)
    I_nd = np.eye(nn * d)
    D_matrix = Dj @ np.eye(g)  

    for i1 in range(g):
        Dbej1 = D_matrix[:, i1]
        Abej1 = Aj @ np.kron(I_nd, np.eye(g)[:, i1].reshape(g, 1))  
        diag_Abej1 = np.diag(Abej1)

        for i2 in range(g):
            # ticnn1 = time()
            Dbej2 = D_matrix[:, i2]
            Abej2 = Aj @ np.kron(I_nd, np.eye(g)[:, i2].reshape(g, 1))
            diag_Abej2 = np.diag(Abej2)

            VQ1 = Dbej1.T @ HI @ Dbej2
            VQ2 = np.trace(Abej1 @ HI @ (Abej2 + Abej2.T) @ HI)
            VQ3 = mu3z * (np.sum(diag_Abej1 * Dbej2) + np.sum(diag_Abej2 * Dbej1))
            VQ4 = mu4z * np.sum(diag_Abej1 * diag_Abej2)

            Sigma_Q[i1, i2] = (VQ1 + VQ2 + VQ3 + VQ4) / (nn * tau2_est[j]**2)
            # tocnn1 = time()
            # print(tocnn1 - ticnn1)

    tocn1 = time()
    # print(tocn1 - ticn1) 
    # print(Sigma_Q)
    Sigma_Q2 = Sigma2_inv@Sigma_Q@Sigma2_inv
    sig = Sigma2_inv[0,0] + Sigma_Q2[0,0]
    
    lower = rho_est[j] - z_alpha*sig**0.5/nn**0.5
    upper = rho_est[j] + z_alpha*sig**0.5/nn**0.5

    
    return lower,upper

In [ ]:
pip install ray

In [ ]:
pip install -U ipywidgets

In [ ]:
# parallel computation framework
import ray

ray.init(num_cpus=NUM_CPU, ignore_reinit_error=True)

In [24]:
jj = random.randint(0, p)

In [27]:
# Full process
@ray.remote(num_cpus=4) 
def map_fun_fmle(bb):
    
    Y, X, Z = data_generator(n, p, q, d, seed = bb+88)
    n0 = int(0.1*n)
    n1 = n - n0
    WW0 = W[:n0,:n0]
    Y0, X0, Z0 = Y[:n0], X[:n0,:], Z[:n0,:]  
    WW1 = W[n0:,n0:]
    Y1, X1, Z1 = Y[n0:], X[n0:,:], Z[n0:,:]
    # initial estimators
    # ticn = time()
    rho_hi, beta_hi, sigma2_hi, ite_hi = optimize_initial(n1, p, q, WW1, Y1, X1, bb)
    rho_hb, beta_hb, sigma2_hb, ite_hb = optimize_initial(n0, p, q, WW0, Y0, X0, bb)
    # tocn = time()
    # print(tocn - ticn)
    
    #print(beta_hb.shape, X0.shape)
    print('cmle is over')
    # Estimate Zi and M
    Z_h, M = Z_est(n0, p, q, d, WW0, WW1, Y1, X1, rho_hi, beta_hi, Y0, X0, rho_hb, beta_hb)
    
    Z_err_max = np.max(abs(Z_h - Z1))
    Z_err_max1 = np.max(abs(-Z_h - Z1))
    print('Z_err_max:', Z_err_max, 'Z_err_max1:', Z_err_max1)
    if Z_err_max>Z_err_max1:
        Z_h = - Z_h
    if (Z_err_max>10) and (Z_err_max1>10):
        return np.zeros((8,p)),np.zeros(p),np.zeros(p),np.zeros(p)
    
    # FMLE with true Z
    rho_hfZ, beta_hfZ, b_hfZ, tau2_hfZ, ite_hfZ = optimize(n1, p, q, d,WW1, Z1, Y1,X1,bb)
    print('SARX is over')
    
    # FMLE with estimated Z
    rho_hf, beta_hf, b_hf, tau2_hf, ite_hf = optimize(n1, p, q, d, WW1, Z_h, Y1, X1,bb)
    print('fmle is over')

    H_hat = M.T@M/p
    bc_hat = b_hf.T@H_hat
    sigma2_hat = np.zeros(p)
    for j in range(p):
        sigma2_hat[j] = bc_hat[j,:]@bc_hat[j,:] + tau2_hf[j]

    lower_est = np.zeros(p)
    upper_est = np.zeros(p)
    q0 = 10
    
    tic1nj = time()
    lower_est[jj], upper_est[jj] = CI_est_test(jj, n1, p, q0, d, WW1, rho_hf, beta_hf, b_hf, tau2_hf, M, sigma2_hat, Y1, X1, Z_h, 1.96)
    toc1nj = time()
    print(bb, jj, toc1nj - tic1nj) # computation time
    # for j in range(p):
    #     tic1nj = time()
    #     lower_est[j], upper_est[j] = CI_est_new(j, n1, p, q0, d, WW1, rho_hf, beta_hf, b_hf, tau2_hf, M, sigma2_hat, Y1, X1, Z_h, 1.96)
    #     toc1nj = time()
    with open('Results_dep_02/process.txt', 'a') as f1:
        f1.write(str(bb)+', '+str(jj)+', '+str(toc1nj - tic1nj)+', '+str(lower_est[jj])+', '+str(upper_est[jj])+'\n')   

    return np.array([rho_hi, rho_hfZ, rho_hf, sigma2_hi, tau2_hfZ, tau2_hf, lower_est, upper_est]), ite_hi, ite_hfZ, ite_hf

In [26]:
from time import time
from numpy.random import default_rng

In [76]:
BB = 500
tic1 = time()
tasks = [map_fun_fmle.remote(bb) for bb in range(BB)]
results500_p50whole = ray.get(tasks)  # 等待所有任务完成
toc1 = time()
print(toc1 - tic1) # 总的计算时间

In [63]:
ray.shutdown()

In [84]:
# Results
BB = 500
theta_hat_dep_n500_p50_B500 = np.zeros((BB,8,p))
ite_ini = np.zeros((BB,p))
ite_fZin = np.zeros((BB,p))
ite_fin = np.zeros((BB,p))
#tic1 = time()
for bt in range(BB):
    theta_hat_dep_n500_p50_B500[bt,:,:], ite_ini[bt,:],ite_fZin[bt,:], ite_fin[bt,:] = results500_p50whole[bt][0], results500_p50whole[bt][1],results500_p50whole[bt][2],results500_p50whole[bt][3]
#toc1 = time()

In [85]:
ECP = np.zeros(BB)
for b in range(BB):
    ECP[b] = (theta_hat_dep_n500_p50_B500[b,6,jj]<=rho[jj]) & (rho[jj]<=theta_hat_dep_n500_p50_B500[b,7,jj])

In [86]:
np.mean(ECP) 

0.956

In [87]:
cc = list(set(range(BB)).difference(set(np.where(ite_fin==0)[0])))

In [89]:
#cmle
np.nanmean(abs(theta_hat_dep_n500_p50_B500[cc,0,:]-((rho.reshape((p,1))@np.ones((1,BB))).T)[cc,:]))

0.018847861065374283

In [90]:
#SARX with true Z
np.nanmean(abs(theta_hat_dep_n500_p50_B500[cc,1,:]-((rho.reshape((p,1))@np.ones((1,BB))).T)[cc,:]))

0.005853870418486939

In [91]:
#fmle
np.nanmean(abs(theta_hat_dep_n500_p50_B500[cc,2,:]-((rho.reshape((p,1))@np.ones((1,BB))).T)[cc,:]))

0.010496041790395521

In [92]:
# save results
for b in range(BB):
    pd.DataFrame(theta_hat_dep_n500_p50_B500[b,:,:]).to_csv("Results_dep_02/n500_p50_B500/theta_hat_dep_n500_p50_"+str(b)+'_.csv',index=False)